In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml.evaluation import RegressionEvaluator
import pyspark.sql.functions as psf
import pandas as pd
import numpy as np
import csv

spark = SparkSession.builder.appName("project3").getOrCreate()3

In [2]:
# Data_1 =  spark.read.csv("/home/hz2558/1B.csv", inferSchema=True, encoding='utf-8')
# Data_2 =  spark.read.csv("/home/hz2558/2B.csv", inferSchema=True, encoding='utf-8')
# Data_3 =  spark.read.csv("/home/hz2558/3B.csv", inferSchema=True, encoding='utf-8')
# Data_4 =  spark.read.csv("/home/hz2558/4B.csv", inferSchema=True, encoding='utf-8')
# Data_5 =  spark.read.csv("/home/hz2558/5B.csv", inferSchema=True, encoding='utf-8')
Data_m =  spark.read.csv("/home/hz2558/Median.csv", inferSchema=True, encoding='utf-8')
Data_f =  spark.read.csv("/home/hz2558/Feature.csv", inferSchema=True, encoding='utf-8')

In [9]:
dfm = Data_m.select(Data_m._c0, Data_m._c270)
dfm = dfm.toPandas()[1:]
dfm = dfm.sort_values(by=["_c0"])

In [10]:
dff = Data_f.toPandas()[1:]

In [11]:
# col1 = ["Zipcode", "Bedroom"]
col2 = ["Zipcode", "Meanvalue"]
col3 = ["Zipcode", "MarketHealth", "ZHVI", "MoM", "YoY", "Forecast", "DayOnMarket"]

In [12]:
# df.columns = col1
dfm.columns = col2
dff.columns = col3

# df.Zipcode = pd.to_numeric(df.Zipcode, errors='coerce')

dfm.Zipcode = pd.to_numeric(dfm.Zipcode, errors='coerce')
dfm.Meanvalue = pd.to_numeric(dfm.Meanvalue, errors='coerce')

dff.Zipcode = pd.to_numeric(dff.Zipcode, errors='coerce')
dff.MarketHealth = pd.to_numeric(dff.MarketHealth, errors='coerce')
dff.ZHVI = pd.to_numeric(dff.ZHVI, errors='coerce')
dff.MoM = pd.to_numeric(dff.MoM, errors='coerce')
dff.YoY = pd.to_numeric(dff.YoY, errors='coerce')
dff.Forecast = pd.to_numeric(dff.Forecast, errors='coerce')
dff.DayOnMarket = pd.to_numeric(dff.DayOnMarket, errors='coerce')

In [13]:
# Dfb = spark.createDataFrame(df)
Dfm = spark.createDataFrame(dfm)
Dff = spark.createDataFrame(dff)

In [14]:
D = Dfm.alias("Dfm").join(Dff.alias("Dff"), psf.col("Dfm.Zipcode") == psf.col("Dff.Zipcode"))\
    .select(
        psf.col("Dfm.Zipcode"),
        psf.col("Dff.MarketHealth"),
        psf.col("Dff.ZHVI"),
        psf.col("Dff.MoM"),
        psf.col("Dff.YoY"),
        psf.col("Dff.Forecast"),
        psf.col("Dff.DayOnMarket"),
        psf.col("Dfm.Meanvalue")
    )

In [15]:
D.show(3)

+-------+------------------+--------+-----------+-----------+-----------+-----------+---------+
|Zipcode|      MarketHealth|    ZHVI|        MoM|        YoY|   Forecast|DayOnMarket|Meanvalue|
+-------+------------------+--------+-----------+-----------+-----------+-----------+---------+
|   1950|5.9374370679999995|508500.0|0.256309148|4.200819672|0.086884956|       77.0|      334|
|   2453| 6.301939987999999|586400.0|0.153714774|5.600576265|0.092501705|       55.5|      376|
|   7747|       2.589111902|386600.0|0.520020801|6.034009874|0.072421107|      103.0|      205|
+-------+------------------+--------+-----------+-----------+-----------+-----------+---------+
only showing top 3 rows



In [18]:
from pyspark.ml.feature import VectorAssembler
vecAss = VectorAssembler(inputCols=D.columns[:-1], outputCol='features')
df_va = vecAss.transform(D)
training = df_va.select(df_va.Meanvalue, df_va.features).selectExpr("Meanvalue as label", "features as features")
training.show(3) 

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  334|[1950.0,5.9374370...|
|  376|[2453.0,6.3019399...|
|  205|[7747.0,2.5891119...|
+-----+--------------------+
only showing top 3 rows



In [11]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

lrModel = lr.fit(training)
result1 = lrModel.transform(training)
result1.show(5)

trainingSummary1 = lrModel.summary
print("RMSE: %f" % trainingSummary1.rootMeanSquaredError)

+-----+--------------------+------------------+
|label|            features|        prediction|
+-----+--------------------+------------------+
|  334|[1950.0,5.9374370...| 292.1250224039209|
|  376|[2453.0,6.3019399...| 328.9158389526128|
|  205|[7747.0,2.5891119...|223.43535923189478|
|  124|[8075.0,0.4168624...|125.36617457460501|
|  363|[11434.0,7.359871...| 279.4597546403248|
+-----+--------------------+------------------+
only showing top 5 rows

RMSE: 73.150839


In [13]:
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3)

glrModel = glr.fit(training)
result2 = glrModel.transform(training)
result2.show(3)

glr_evaluator = RegressionEvaluator(labelCol='label', metricName="rmse", predictionCol='prediction')
rmse = glr_evaluator.evaluate(result2)
print("RMSE: %f" % rmse)

+-----+--------------------+------------------+
|label|            features|        prediction|
+-----+--------------------+------------------+
|  334|[1950.0,5.9374370...|292.07897828346535|
|  376|[2453.0,6.3019399...|328.51754839969715|
|  205|[7747.0,2.5891119...|223.19646008996182|
+-----+--------------------+------------------+
only showing top 3 rows

RMSE: 73.148067


In [192]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(maxDepth=5, varianceCol="variance")
dt_model = dt.fit(training)

result3 = dt_model.transform(training)
result3.show(5)

dt_evaluator = RegressionEvaluator(labelCol='label', metricName="rmse", predictionCol='prediction')
rmse = dt_evaluator.evaluate(result3)
print("RMSE: %f" % rmse)

+-----+--------------------+------------------+------------------+
|label|            features|        prediction|          variance|
+-----+--------------------+------------------+------------------+
|  334|[1950.0,5.9374370...|  274.912037037037| 6858.344114368997|
|  376|[2453.0,6.3019399...|383.65263157894736|12406.114422899354|
|  205|[7747.0,2.5891119...|223.71428571428572|3319.6854003139742|
|  124|[8075.0,0.4168624...|132.98601398601397| 543.4630911046984|
|  363|[11434.0,7.359871...|223.71428571428572|3319.6854003139742|
+-----+--------------------+------------------+------------------+
only showing top 5 rows

RMSE: 65.626966


In [193]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(maxIter=10, maxDepth=3)
gbt_model = gbt.fit(training)

result4 = gbt_model.transform(training)
result4.show(5)

gbt_evaluator = RegressionEvaluator(labelCol='label', metricName="rmse", predictionCol='prediction')
rmse = gbt_evaluator.evaluate(result4)
print("RMSE: %f" % rmse)

+-----+--------------------+------------------+
|label|            features|        prediction|
+-----+--------------------+------------------+
|  334|[1950.0,5.9374370...|308.25388452217135|
|  376|[2453.0,6.3019399...| 386.7227771658733|
|  205|[7747.0,2.5891119...|209.15181812451144|
|  124|[8075.0,0.4168624...|142.16285264233656|
|  363|[11434.0,7.359871...|  227.352679377431|
+-----+--------------------+------------------+
only showing top 5 rows

RMSE: 68.442685


In [194]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(numTrees=10, maxDepth=5, seed=101)
rf_model = rf.fit(training)

result5 = rf_model.transform(training)
result5.show(5)

rf_evaluator = RegressionEvaluator(labelCol='label', metricName="rmse", predictionCol='prediction')
rmse = rf_evaluator.evaluate(result5)
print("RMSE: %f" % rmse)

+-----+--------------------+------------------+
|label|            features|        prediction|
+-----+--------------------+------------------+
|  334|[1950.0,5.9374370...|243.34061061104694|
|  376|[2453.0,6.3019399...|331.43454699769745|
|  205|[7747.0,2.5891119...|233.60136260840076|
|  124|[8075.0,0.4168624...|138.42802726691647|
|  363|[11434.0,7.359871...|294.25538346496904|
+-----+--------------------+------------------+
only showing top 5 rows

RMSE: 66.639252


In [195]:
s = result3.toPandas()

In [201]:
result = D.toPandas()
result["predictions"] = s["prediction"]

In [206]:
process = np.array(result)
process

array([[1.95000000e+03, 5.93743707e+00, 5.08500000e+05, ...,
        7.70000000e+01, 3.34000000e+02, 2.74912037e+02],
       [2.45300000e+03, 6.30193999e+00, 5.86400000e+05, ...,
        5.55000000e+01, 3.76000000e+02, 3.83652632e+02],
       [7.74700000e+03, 2.58911190e+00, 3.86600000e+05, ...,
        1.03000000e+02, 2.05000000e+02, 2.23714286e+02],
       ...,
       [9.77540000e+04, 7.06920857e+00, 2.24800000e+05, ...,
        8.60000000e+01, 1.48000000e+02, 1.72748219e+02],
       [9.86830000e+04, 9.15956233e+00, 3.69500000e+05, ...,
        5.00000000e+01, 2.07000000e+02, 2.10801170e+02],
       [9.92040000e+04, 9.57709606e+00, 2.35400000e+05, ...,
        5.00000000e+01, 2.09000000e+02, 1.72748219e+02]])

In [264]:
process

array([[1.95000000e+03, 5.93743707e+00, 5.08500000e+05, ...,
        7.70000000e+01, 3.34000000e+02, 2.74912037e+02],
       [2.45300000e+03, 6.30193999e+00, 5.86400000e+05, ...,
        5.55000000e+01, 3.76000000e+02, 3.83652632e+02],
       [7.74700000e+03, 2.58911190e+00, 3.86600000e+05, ...,
        1.03000000e+02, 2.05000000e+02, 2.23714286e+02],
       ...,
       [9.77540000e+04, 7.06920857e+00, 2.24800000e+05, ...,
        8.60000000e+01, 1.48000000e+02, 1.72748219e+02],
       [9.86830000e+04, 9.15956233e+00, 3.69500000e+05, ...,
        5.00000000e+01, 2.07000000e+02, 2.10801170e+02],
       [9.92040000e+04, 9.57709606e+00, 2.35400000e+05, ...,
        5.00000000e+01, 2.09000000e+02, 1.72748219e+02]])

In [265]:
D

DataFrame[Zipcode: bigint, MarketHealth: double, ZHVI: double, MoM: double, YoY: double, Forecast: double, DayOnMarket: double, Meanvalue: bigint]

In [269]:
Zip = list(process[:, 0])
for i in range(len(Zip)):
    Zip[i] = str(int(Zip[i]))
    if len(Zip[i]) == 4:
        Zip[i] = "0" + Zip[i]
        
Zip = pd.DataFrame(Zip)
Mar = pd.DataFrame(np.around(process[:, 1], decimals=2))
MoM = pd.DataFrame(np.around(process[:, 3], decimals=2))
YoY = pd.DataFrame(np.around(process[:, 4], decimals=2))
Day = pd.DataFrame(np.around(process[:, 6], decimals=1))
Pre = pd.DataFrame(np.around(process[:, 8], decimals=2))

In [270]:
Zip["Mar"] = Mar
Zip["Mom"] = MoM
Zip["YoY"] = YoY
Zip["Day"] = Day
Zip["Pre"] = Pre

In [271]:
Zip.columns = ["Zipcode", "MarketHealth", "MoM", "YoY", "DayOnMarket", "Prediction"]
Zip

,Zipcode,MarketHealth,MoM,YoY,DayOnMarket,Prediction
0,01950,5.94,0.26,4.20,77.0,274.91
1,02453,6.30,0.15,5.60,55.5,383.65
2,07747,2.59,0.52,6.03,103.0,223.71
3,08075,0.42,0.25,8.58,104.0,132.99
4,11434,7.36,0.91,10.16,171.0,223.71
5,15057,3.80,-0.66,3.39,88.5,132.99
6,15237,4.44,0.00,4.12,63.5,152.52
7,15437,9.73,1.73,7.25,109.5,106.20
8,17043,2.48,0.12,5.37,75.0,106.20
9,17048,0.91,-0.96,11.46,74.5,72.59


In [272]:
Zip.to_csv("Median_prediction.csv", index=False, sep=",")